In [1]:
# ----------------
# import statements
# ----------------
import sqlite3
from pathlib import Path
import pandas as pd

In [2]:
# ----------------
# global variables
# ----------------
ZERO_ADDRESS =	'0x0000000000000000000000000000000000000000'
DB_CONN = sqlite3.connect('./db/inspector.db')
DB_CURS = DB_CONN.cursor()
DEBUG = False

In [14]:
def debug_print(*args):
	if DEBUG == True:
		print(args)

## Initialize the local database

In [4]:
# --------------------------------------------------------------------------------------------------
# create all database tables if they don't already exist
def initialize_db_tables(db_curs):

	# SQL statement to create transactions table
	sql_create_transactions_table = '''
	CREATE TABLE IF NOT EXISTS transactions(
		contract_id INT,
		block_num INT,
		tx_hash TEXT,
		sender TEXT,
		receiver TEXT,
		tx_timestamp TEXT,
		tokens REAL
	)
	'''
	db_curs.execute(sql_create_transactions_table)

	# SQL statement to create wallets table
	sql_create_wallet_table = '''
	CREATE TABLE IF NOT EXISTS wallets(
		wallet TEXT,
		tokenid INT,
		tokens REAL,
		last_tx TEXT,
		UNIQUE (wallet)
	)
	'''
	db_curs.execute(sql_create_wallet_table)


In [5]:
initialize_db_tables(DB_CURS)

## Populate the database tables

In [6]:
def populate_db_tables(db_conn, db_curs):
	
	# --------------------------------------------------------------------------------------------------
	# read the transactions csv and save them to the database

	# setting filepath for raw transactions csv
	raw_tx_filepath = Path('./data/walletmapping.csv')

	# saving the csv header as a list
	raw_tx_header = ["contract_id" , "block_num", "tx_hash", "sender", "receiver", "tx_timestamp", "tokens"]

	# read the transaction CSV into a dataframe for easy manipulation
	raw_tx_transactions_df = pd.read_csv(
		filepath_or_buffer =	raw_tx_filepath,
		header =	0,
		names  =	raw_tx_header
	)

	# debug_print(raw_tx_transactions_df)

	# write all transactions to the database
	# this currently REPLACES all transactions in the 'transactions' table
	raw_tx_transactions_df.to_sql(
		name =	"transactions",
		con = 	db_conn,
		if_exists="replace",
		index=False 
	)
	
	# --------------------------------------------------------------------------------------------------
	# populate the wallets table with all unique wallet addresses (senders and receivers)
	# select the sender column
	# union the sender column with the receiver column
	# populate the wallets table (wallet) column with the results
	sql_populate_wallets_table = '''
	INSERT OR IGNORE INTO wallets (wallet)
	SELECT sender FROM transactions t
	UNION 
	SELECT receiver FROM transactions t
	'''
	db_curs.execute(sql_populate_wallets_table)
	db_conn.commit()


In [7]:
populate_db_tables(DB_CONN, DB_CURS)

## Generate a sorted transaction dataframe

In [8]:
def create_sorted_tx(db_conn):
	# --------------------------------------------------------------------------------------------------
	# build a dataframe of all transactions sorted by transaction date

	# SQL code to get list of transactions by transaction date
	sql_select_sort_tx_by_date = '''
	SELECT
		tx_timestamp,
		sender,
		receiver,
		tokens
	FROM transactions
	ORDER BY tx_timestamp ASC
	'''
	# df index
	sorted_tx_index = 'tx_timestamp'
	
	# df columns
	sorted_tx_cols  = ['sender', 'receiver', 'tokens']
	
	# load sorted transactions into pandas dataframe
	sorted_tx_df = pd.read_sql(
		sql =	sql_select_sort_tx_by_date, 
		con =	DB_CONN,
		parse_dates =	True,
		index_col =	sorted_tx_index,
		columns   =	sorted_tx_cols
	)
	
	return sorted_tx_df


In [9]:
sorted_tx_df = create_sorted_tx(DB_CONN)

display(sorted_tx_df)

,sender,receiver,tokens
tx_timestamp,,,
2021-07-10 20:55:30,0x0000000000000000000000000000000000000000,0xf0e3ea754d038b979cd0124e2f1a4bf44f32746a,1.000000e+10
2021-07-27 19:07:24,0xf0e3ea754d038b979cd0124e2f1a4bf44f32746a,0x9aea240869e1e9935a247520ba67d29c08ede443,0.000000e+00
2021-08-12 13:42:37,0xf0e3ea754d038b979cd0124e2f1a4bf44f32746a,0x2ea91d600364d77053ca0161424298be6b0379aa,3.000000e+01
2021-08-13 07:17:39,0x2ea91d600364d77053ca0161424298be6b0379aa,0x4f5e789977d732457b87835cf594cb41f7aafd96,5.000000e+00
2021-08-18 08:06:02,0x2ea91d600364d77053ca0161424298be6b0379aa,0xa7350296c6f3f3f0fc9aa1752b2bdcfaba274c16,5.000000e+00
...,...,...,...
2022-10-21 06:40:23,0xdac9b2c7a5a8e2163b4161e34c2148e11b51fcc7,0xdd51121d1efc398b4c09fd0cb84d79ae2c923fc9,1.673936e+06
2022-10-21 06:40:23,0x1082127211bda2f0ed1a3d50a1ab5d0d5895957b,0xdac9b2c7a5a8e2163b4161e34c2148e11b51fcc7,1.259931e+06
2022-10-21 06:51:23,0xdd51121d1efc398b4c09fd0cb84d79ae2c923fc9,0xdac9b2c7a5a8e2163b4161e34c2148e11b51fcc7,5.118656e+05


In [17]:
def reset_wallet_tokens(db_conn, db_curs):
	
	# --------------------------------------------------------------------------------------------------
	# resetting wallet database token counts
	sql_zero_wallet_tokens = "UPDATE OR FAIL wallets SET tokens=?"
	db_curs.execute(sql_zero_wallet_tokens, (None,))
	db_conn.commit()
	

In [18]:
# helper function to reset wallet tokens counts if necessary
reset_wallet_tokens(DB_CONN, DB_CURS)

## Generate wallet token totals

In [19]:
def calculate_wallet_tokens(db_conn, db_curs, df):
	
	# --------------------------------------------------------------------------------------------------
	# calculate wallet token totals based on the transactions in transaction table

	# add code to be able to time slice transactions on a day-by-day basis

	# all SQL transactions necessary to update the wallets table
	sql_get_wallet_tokens = "SELECT tokens FROM wallets WHERE wallet=?"
	sql_get_wallet_last_tx = "SELECT last_tx FROM wallets WHERE wallet=?"
	sql_update_wallet_tokens = "UPDATE OR FAIL wallets SET tokens=? WHERE wallet=?"
	sql_update_wallet_last_tx = "UPDATE OR FAIL wallets SET last_tx=? WHERE wallet=?"

	for row in df.iterrows():

		debug_print('--------------------------------')
		debug_print('row index', row[0])
		current_tx = row[0]
		# ----------------
		# dt_current_tx = pd.to_datetime(current_tx)
		# str_current_tx = dt_current_tx.strftime('%Y-%m-%d %X')
		# ----------------

		# ----------------------------------------------------------
		# get 'last_tx' from sender and receiver wallets

		# get the sender wallet's last transaction
		DB_CURS.execute(sql_get_wallet_last_tx, (row[1].sender,))
		last_tx_sender = DB_CURS.fetchone()[0]
		dt_last_tx_sender = pd.to_datetime(last_tx_sender)

		# get the receiver wallet's last transaction
		DB_CURS.execute(sql_get_wallet_last_tx, (row[1].receiver,))
		last_tx_receiver = DB_CURS.fetchone()[0]
		dt_last_tx_receiver = pd.to_datetime(last_tx_receiver)

		# ----------------------------------------------------------
		# TODO if the current index (row[0]) is less than the sender or receiver's last transaction skip this transaction

		debug_print('----')
		debug_print('sender', row[1].sender)

		# ----------------------------------------------------------
		# if not ZERO ADDRESS then subtract tokens from the sender wallet
		if not row[1].sender == ZERO_ADDRESS:

			# get the sender wallet's current token count from the database
			DB_CURS.execute(sql_get_wallet_tokens, (row[1].sender,))
			# fetch only the tokens from the response
			sender_tokens = DB_CURS.fetchone()[0]
			debug_print('sender wallet tokens', sender_tokens)

			# checking to see if the sender wallet's tokens are Null / None
			if sender_tokens == None:
				# if so, set the wallet to zero, however this allows us to have wallets with negative token balances
				sender_tokens = 0
				debug_print('sender wallet tokens None -> 0')
				debug_print('sender wallet tokens', receiver_tokens)

			# remove the tokens sent from the sender wallet
			sender_tokens -= row[1].tokens
			debug_print('sender wallet tokens', sender_tokens)
		# ----------------------------------------------------------
		else:
			debug_print('ZERO ADDRESS - skipping subtraction')

		debug_print('----')
		debug_print('receiver', row[1].receiver)

		# ----------------------------------------------------------
		# get the receiver wallet's current token count from the database
		DB_CURS.execute(sql_get_wallet_tokens, (row[1].receiver,))
		# fetch only the tokens from the response
		receiver_tokens = DB_CURS.fetchone()[0]
		debug_print('receiver wallet tokens', receiver_tokens)

		# ----------------------------------------------------------
		# checking to see if the receiver wallet's tokens are Null / None
		if receiver_tokens == None:
			# if so, set the wallet to zero
			receiver_tokens = 0
			debug_print('receiver wallet tokens None -> 0')
			debug_print('receiver wallet tokens', receiver_tokens)

		# ----------------------------------------------------------
		# add the tokens sent to the receiver wallet
		receiver_tokens += row[1].tokens
		debug_print('receiver wallet tokens', receiver_tokens)
		debug_print('----')

		# ----------------------------------------------------------
		# if not the ZERO ADDRESS then update the sender's wallet with the updated token count
		if not row[1].sender == ZERO_ADDRESS:
			# execute the transaction to update the sender's token count
			db_curs.execute(sql_update_wallet_tokens, (sender_tokens, row[1].sender))

		# execute the transaction to update the receiver's token count
		db_curs.execute(sql_update_wallet_tokens, (receiver_tokens, row[1].receiver))

		# TODO update 'last_tx' field in wallets table for the receiver wallet
		db_curs.execute(sql_update_wallet_last_tx, (row[0], row[1].sender))
		db_curs.execute(sql_update_wallet_last_tx, (row[0], row[1].receiver))

		# commit the three (or four) transactions executed on the database immediately above
		db_conn.commit()


In [20]:
calculate_wallet_tokens(DB_CONN, DB_CURS, sorted_tx_df)